# Aprendizaje Reforzado
## El laberinto del muffintauro


1. Programe el ambiente que simule el laberinto del muffintaurio, debe construir una función step(state, action) que reciba un estado y una acción, y que devuelva el estado siguiente y la recompensa. nota: este ambiente es determinístico, por lo que la probabilidad de transición a un determinado s' es 1

In [1]:

nodo = 1  # Elegir el nodo de partida
muffins_max = 10
muffins = 6  # Elegir la cantidad de muffins que lleva consigo
state = (nodo, muffins)
acciones_posibles = [(0, 1), (0, 2), (1, 0), (1, 3), (2, 0), (2, 3)]
destino = 3 #Elegir el nodo de destino
action = (nodo, destino) 

# Dependiendo el nodo al que vaya calcula el reward o costo de este
def rewards(state, action): 
    if state[1] == 0:
        return -42
    if action[1] == 1:
        return 0
    if action[1] == 2:
        return -1
    if action[1] == 3:
        return 6 - abs(state[1] - 6)

# Actúa en los casos en los que se quiten u obtengan muffins en el recorrido
def manejo_de_muffins(action):
    if action[1] == 1:
        return -2
    elif action[1] == 2:
        return 2
    else:
        return 0
    
# Función step
def step(state, action):
    if action in acciones_posibles and state[0] == action[0]:
        r_destino = rewards(state, action)
        nuevo_muffins = state[1] + manejo_de_muffins(action)
        new_state = (action[1], max(0, min(muffins_max, nuevo_muffins))) 
        print(f"El estado siguiente es el nodo {new_state[0]}, con {new_state[1]} muffins y su recompensa es {r_destino}")
    else:
        print("Acción no válida")

# Ejecuta la función
step(state, action)


            



El estado siguiente es el nodo 3, con 6 muffins y su recompensa es 6


2. Encuentre la política optima usando Value Iteration, para esto lea el capítulo 4.4. del libro de Sutton y Barto e implemente el algoritmo. Use una tolerancia tol = 0.1

In [2]:
# La estructura del laberinto
nodos = [0, 1, 2, 3]
acciones = [(0, 1), (0, 2), (1, 0), (1, 3), (2, 0), (2, 3)]
gamma = 0.9  # Factor de descuento
tol = 0.1  # Umbral para la convergencia
muffins_max = 10  # Tope de cantidad de muffins

# Inicializar el grafo como un diccionario de listas
laberinto = {
    0: [1, 2],
    1: [0, 3],
    2: [0, 3],
    3: []
}

# Función de recompensa
def calcular_recompensa(state, destino):
    nodo, muffins = state
    if muffins <= 0:
        return -42  # Penalización por quedarse sin muffins
    if destino == 2:
        return -1  # Penalización por pasar por el nodo 2
    if destino == 3:
        return 6 - abs(muffins - 6)  # Recompensa al llegar al nodo final
    return 0  # Recompensa por otros movimientos

# Función para calcular los muffins después de una acción
def calcular_muffins(state, destino):
    nodo, muffins = state
    if destino == 1:
        muffins -= 2
    elif destino == 2:
        muffins += 2
    return max(0, min(muffins_max, muffins))  # Asegurarse de que los muffins estén en el rango permitido

# Inicializar los valores de los estados a cero
valores = {(nodo, muffins): 0 for nodo in nodos for muffins in range(muffins_max + 1)}

# Inicializar las políticas
politicas = {(nodo, muffins): None for nodo in nodos for muffins in range(muffins_max + 1)}

# Función de iteración de valores
def iteracion_de_valores(laberinto, gamma, tol):
    while True:
        delta = 0
        nuevos_valores = valores.copy()
        for nodo in nodos:
            for muffins in range(muffins_max + 1):
                state = (nodo, muffins)
                if nodo == 3:
                    continue  # No hay necesidad de actualizar el nodo final

                max_valor = float('-inf')
                mejor_accion = None
                for destino in laberinto[nodo]:
                    recompensa = calcular_recompensa(state, destino)
                    nuevo_muffins = calcular_muffins(state, destino)

                    # Valor descontado del siguiente estado
                    nuevo_estado = (destino, nuevo_muffins)
                    valor_estado_siguiente = valores[nuevo_estado]
                    valor_actual = recompensa + gamma * valor_estado_siguiente

                    if valor_actual > max_valor:
                        max_valor = valor_actual
                        mejor_accion = destino

                nuevos_valores[state] = max_valor
                politicas[state] = mejor_accion
                delta = max(delta, abs(valores[state] - nuevos_valores[state]))

        valores.update(nuevos_valores)
        if delta < tol:
            break

    return valores, politicas

# Ejecutar iteración de valores
valores_optimos, politicas_optimas = iteracion_de_valores(laberinto, gamma, tol)

# Imprimir las políticas óptimas de los estados, excluyendo el nodo 3
for estado, accion in sorted(politicas_optimas.items()):
    if estado[0] == 3:
        continue  # No imprimir políticas para el nodo 3, ya que terminó el recorrido
    print(f"La política óptima del nodo {estado[0]} llevando {estado[1]} muffins es ir al nodo {accion}")



La política óptima del nodo 0 llevando 0 muffins es ir al nodo 2
La política óptima del nodo 0 llevando 1 muffins es ir al nodo 2
La política óptima del nodo 0 llevando 2 muffins es ir al nodo 2
La política óptima del nodo 0 llevando 3 muffins es ir al nodo 2
La política óptima del nodo 0 llevando 4 muffins es ir al nodo 2
La política óptima del nodo 0 llevando 5 muffins es ir al nodo 2
La política óptima del nodo 0 llevando 6 muffins es ir al nodo 1
La política óptima del nodo 0 llevando 7 muffins es ir al nodo 1
La política óptima del nodo 0 llevando 8 muffins es ir al nodo 1
La política óptima del nodo 0 llevando 9 muffins es ir al nodo 1
La política óptima del nodo 0 llevando 10 muffins es ir al nodo 1
La política óptima del nodo 1 llevando 0 muffins es ir al nodo 3
La política óptima del nodo 1 llevando 1 muffins es ir al nodo 0
La política óptima del nodo 1 llevando 2 muffins es ir al nodo 0
La política óptima del nodo 1 llevando 3 muffins es ir al nodo 0
La política óptima del n

3. Imprima una tabla de nodo vs cantidad de muffins que muestre el valor de cada estado.

In [5]:
import pandas as pd

# La estructura del laberinto
nodos = [0, 1, 2, 3]
acciones = [(0, 1), (0, 2), (1, 0), (1, 3), (2, 0), (2, 3)]
gamma = 0.9  # Factor de descuento
tol = 0.1  # Umbral para la convergencia
muffins_max = 10  # Topde de cantidad de muffins

# Inicializar el grafo como un diccionario de listas
laberinto = {
    0: [1, 2],
    1: [0, 3],
    2: [0, 3],
    3: []
}

# Función de recompensa
def calcular_recompensa(state, destino):
    nodo, muffins = state
    if muffins <= 0:
        return -42  # Penalización por quedarse sin muffins
    if destino == 2:
        return -1  # Penalización por pasar por el nodo 2
    if destino == 3:
        return 6 - abs(muffins - 6)  # Recompensa al llegar al nodo final
    return 0  # Recompensa por otros movimientos

# Función para calcular los muffins después de una acción
def calcular_muffins(state, destino):
    nodo, muffins = state
    if destino == 1:
        muffins -= 2
    elif destino == 2:
        muffins += 2
    return max(0, min(muffins_max, muffins))  # Asegurarse de que los muffins estén en el rango permitido

# Inicializar los valores de los estados a cero
valores = {(nodo, muffins): 0 for nodo in nodos for muffins in range(muffins_max + 1)}

# Función de iteración de valores
def iteracion_de_valores(laberinto, gamma, tol):
    while True:
        delta = 0
        nuevos_valores = valores.copy()
        for nodo in nodos:
            for muffins in range(muffins_max + 1):
                state = (nodo, muffins)
                if nodo == 3:
                    continue  # No hay necesidad de actualizar el nodo final

                max_valor = float('-inf')
                for destino in laberinto[nodo]:
                    recompensa = calcular_recompensa(state, destino)
                    nuevo_muffins = calcular_muffins(state, destino)

                    # Valor descontado del siguiente estado
                    nuevo_estado = (destino, nuevo_muffins)
                    valor_estado_siguiente = valores[nuevo_estado]
                    valor_actual = recompensa + gamma * valor_estado_siguiente
                    # Como es determinístico no se lo multiplica por la probabilidad (esta es 1)

                    max_valor = max(max_valor, valor_actual)

                nuevos_valores[state] = max_valor
                delta = max(delta, abs(valores[state] - nuevos_valores[state]))

        valores.update(nuevos_valores)
        if delta < tol:
            break

    return valores

# Ejecutar iteración de valores
valores_optimos = iteracion_de_valores(laberinto, gamma, tol)

# Crear una tabla con pandas
data = []
for (nodo, muffins), valor in valores_optimos.items():
    data.append([nodo, muffins, valor])

df = pd.DataFrame(data, columns=["Nodo", "Muffins", "Valor"])

df = df[df["Nodo"] != 3] # Evito imprimir la fila nula asociada al nodo terminal

pivot_table = df.pivot(index="Nodo", columns="Muffins", values="Valor")

# Imprimir la tabla
print("Cada estado se compone por el nodo actual y la cantidad de muffins. Los valores de cada estado son: ")
print(pivot_table)


Cada estado se compone por el nodo actual y la cantidad de muffins. Los valores de cada estado son: 
Muffins      0       1     2     3    4    5    6    7     8     9       10
Nodo                                                                       
0       -39.894  1.8350  2.60  3.50  4.4  3.5  3.6  4.5  5.40  4.50  4.3740
1       -42.000  1.6515  2.34  3.15  4.0  5.0  6.0  5.0  4.86  4.05  3.9366
2       -42.000  1.6515  2.34  3.15  4.0  5.0  6.0  5.0  4.86  4.05  3.9366


4. Imprima una tabla de nodo vs cantidad de muffins que muestre la política optima para cada estado.

In [2]:
import pandas as pd

# La estructura del laberinto
nodos = [0, 1, 2, 3]
acciones = [(0, 1), (0, 2), (1, 0), (1, 3), (2, 0), (2, 3)]
gamma = 0.9  # Factor de descuento
tol = 0.1  # Umbral para la convergencia
muffins_max = 10  # Tope de cantidad de muffins

# Inicializar el grafo como un diccionario de listas
laberinto = {
    0: [1, 2],
    1: [0, 3],
    2: [0, 3],
    3: []
}

# Función de recompensa
def calcular_recompensa(state, destino):
    nodo, muffins = state
    if muffins <= 0:
        return -42  # Penalización por quedarse sin muffins
    if destino == 2:
        return -1  # Penalización por pasar por el nodo 2
    if destino == 3:
        return 6 - abs(muffins - 6)  # Recompensa al llegar al nodo final
    return 0  # Recompensa por otros movimientos

# Función para calcular los muffins después de una acción
def calcular_muffins(state, destino):
    nodo, muffins = state
    if destino == 1:
        muffins -= 2
    elif destino == 2:
        muffins += 2
    return max(0, min(muffins_max, muffins))  # Asegurarse de que los muffins estén en el rango permitido

# Inicializar los valores de los estados a cero
valores = {(nodo, muffins): 0 for nodo in nodos for muffins in range(muffins_max + 1)}

# Inicializar las políticas
politicas = {(nodo, muffins): None for nodo in nodos for muffins in range(muffins_max + 1)}

# Función de iteración de valores
def iteracion_de_valores(laberinto, gamma, tol):
    while True:
        delta = 0
        nuevos_valores = valores.copy()
        for nodo in nodos:
            for muffins in range(muffins_max + 1):
                state = (nodo, muffins)
                if nodo == 3:
                    continue  # No hay necesidad de actualizar el nodo final

                max_valor = float('-inf')
                mejor_accion = None
                for destino in laberinto[nodo]:
                    recompensa = calcular_recompensa(state, destino)
                    nuevo_muffins = calcular_muffins(state, destino)

                    # Valor descontado del siguiente estado
                    nuevo_estado = (destino, nuevo_muffins)
                    valor_estado_siguiente = valores[nuevo_estado]
                    valor_actual = recompensa + gamma * valor_estado_siguiente

                    if valor_actual > max_valor:
                        max_valor = valor_actual
                        mejor_accion = destino

                nuevos_valores[state] = max_valor
                politicas[state] = mejor_accion
                delta = max(delta, abs(valores[state] - nuevos_valores[state]))

        valores.update(nuevos_valores)
        if delta < tol:
            break

    return valores, politicas

# Ejecutar iteración de valores
valores_optimos, politicas_optimas = iteracion_de_valores(laberinto, gamma, tol)

# Crear una tabla con pandas
data = []
for (nodo, muffins), accion in politicas_optimas.items():
    if accion is not None:
        data.append([nodo, muffins, int(accion)])

# Crear un DataFrame
df = pd.DataFrame(data, columns=["Nodo", "Muffins", "Política Óptima"])

# Filtrar el DataFrame para eliminar la fila con el nodo 3 porque ahí termina el recorrido
df = df[df["Nodo"] != 3]

# Pivotar la tabla
pivot_table = df.pivot(index="Nodo", columns="Muffins", values="Política Óptima")

# Imprimir la tabla
print("La política óptima de cada estado no terminal es:")
print(pivot_table)


La política óptima de cada estado no terminal es:
Muffins  0   1   2   3   4   5   6   7   8   9   10
Nodo                                               
0         2   2   2   2   2   2   1   1   1   1   1
1         3   0   0   0   3   3   3   3   0   0   0
2         3   0   0   0   3   3   3   3   0   0   0


5. Haga un análisis cualitativo de la política optima. ¿Tiene sentido?¿Es intuitiva?¿Es la que esperaba?



La mayor recompensa que se puede obtener en el recorrido es la que se consigue cuando se llega a la casa de la abuela con una cierta cantidad de muffins. El mejor caso es llegar con 6 muffins, ya que ahi la recompensa sera de 6 monedas. Luego se pueden obtener 5 monedas si se llega con 5 o 7 muffins, 4 monedas si se entregan 4 o 8 muffins y asi sucesivamente. Mientras más se aleja el número de muffins del número 6, menor es la recompensa al terminar el recorrido. Es por esto que la política óptima de estar en el nodo 1 o 2 con 4, 5, 6 y 7 muffins es terminar el recorrido, ya que ahí la recompensa va a ser alta. Pero, en el caso de estar en alguno de estos dos nodos y llevar 8 muffins, aún si la recompensa de llevar 8 muffins es igual a la de 4 (y en esta ocación la política óptima es terminar el recorrido), en este caso existe una mejor opción que ir al nodo 3. Esto se debe a que si se vuelve al nodo 0 para despues ir al nodo 1 y perder 2 muffins, como indican sus políticas óptimas, se puede terminar el recorrido con 6 muffins y sin ningún costo adicional, obtener la mayor recompensa posible. Esto tiene mucho sentido, ya que sin ninguna recompensa negativa solo se pueden perden muffins en el camino, tener 8 muffins en el nodo 1 o 2 es mejor que tener 4 en estos, aún si su recompensa por terminar en ese instante sea la misma. La política óptima hace exactamente esto, para cuidar las recompensas de cada estado. 

Por otro lado, cuando el laberinto se empieza con menos de 6 muffins, la política óptima indica que es mejor ir para el nodo 2, ya que en este es posible aumentar la cantidad de muffins y terminar el recorrido con una cantidad más cercana a 6 muffins. Aún si con el paso por el nodo 2 no se llega a un número cercano al 6, la política óptima indica que se debe volver al nodo inicial para volver a pasar por el nodo 2 y así seguir aum,entando la cantidad de muffins. Esto tiene mucho sentido, porque aún si aumentando la cantidad de muffins pierden una moneda cada vez, este costo sigue siendo menor a la recompensa por terminar con 5, 6 o 7 muffins. 

6. Mirando la tabla de valores, ¿Con cuantos muffins conviene empezar el dia?¿Depende del factor de descuento?

Observando la tabla de valores, conviene arrancar el día con 8 muffins, y de ahí seguir conm la política óptima. Esta consiste en ir al nodo 1 y sin ningún costo perder 2 muffins para luego terminar el recorrido con 6 muffins y una recompensa de 6 monedas. El valor de arrancar el recorrido con 8 muffins es mucho mayor a cualquier otra cantidad. Aún probando con distintos valores para $\gamma$ el mayor valor al estar en el nodo 1 sigue perteneciendo al que posee 8 muffins.


